In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld

URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_event_all = ld.getCOOs()
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

ICM_length_all_5bal = ld.getICMlength('5bal')
ICM_length_all_3bal = ld.getICMlength('3bal')
ICM_length_all_5km = ld.getICMlength('5km')
ICM_length_all_3km = ld.getICMlength('5km')

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13648 ( 0.0%) Users that have less than 1 test interactions


In [4]:
import numpy as np
from scipy.sparse import *
R_F = np.zeros((13658,18059))
R_F[:13650,:] = URM_train.todense() * 0.7
R_F[13650:,:] = ICM_genre_all.todense().T * 0.3
R_F = coo_matrix(R_F)

R_F

<13658x18059 sparse matrix of type '<class 'numpy.float64'>'
	with 4246522 stored elements in COOrdinate format>

In [3]:
import os

output_folder_path = "../result_experiments/S_SLIM_ElasticNet_genre_subgenre_channel_5km_2"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 20  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [10]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "l1_ratio": Real(low = 1e-5, high = 1.0, prior = 'log-uniform'),
    "topK": Integer(10, 1000),
    "alpha": Real(low = 1e-3, high = 1.0, prior = 'uniform'),
    "workers": Categorical([10]),
}

In [5]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = MultiThreadSLIM_SLIMElasticNetRecommender

# hyperparameterSearch = SearchBayesianSkopt(recommender_class, evaluator_validation=evaluator_validation)

In [12]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [R_F],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}, # Additiona hyperparameters for the fit function
)

In [13]:
if __name__ == '__main__':
    hyperparameterSearch.search(recommender_input_args,
                           hyperparameter_search_space = hyperparameters_range_dictionary,
                           n_cases = n_cases,
                           n_random_starts = n_random_starts,
                           output_folder_path = output_folder_path,
                           output_file_name_root = recommender_class.RECOMMENDER_NAME,
                           metric_to_optimize = metric_to_optimize,
                           cutoff_to_optimize = cutoff_to_optimize,
                          )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 0.009399338335277865, 'topK': 426, 'alpha': 0.45589827291924323, 'workers': 10}


 25%|██████████████████▉                                                          | 4456/18059 [02:22<05:52, 38.56it/s]


 50%|██████████████████████████████████████▏                                      | 8968/18059 [04:40<04:20, 34.92it/s]


 74%|████████████████████████████████████████████████████████▌                   | 13448/18059 [06:54<02:13, 34.63it/s]


100%|███████████████████████████████████████████████████████████████████████████▉| 18032/18059 [09:13<00:00, 75.61it/s]

EvaluatorHoldout: Processed 13648 (100.0%) in 10.12 sec. Users per second: 1349
SearchBayesianSkopt: New best config found. Config 0: {'l1_ratio': 0.009399338335277865, 'topK': 426, 'alpha': 0.45589827291924323, 'workers': 10} - results: PRECISION: 0.3681565, PRECISION_RECALL_MIN_DEN: 0.3695762, RECALL: 0.0641241, MAP: 0.2270584, MAP_MIN_DEN: 0.2277995, MRR: 0.6361518, NDCG: 0.3873062, F1: 0.1092239, HIT_RATE: 0.9608734, ARHR_ALL_HITS: 1.1807189, NOVELTY: 0.0052583, AVERAGE_POPULARITY: 0.6803493, DIVERSITY_MEAN_INTER_LIST: 0.8038287, DIVERSITY_HERFINDAHL: 0.9803770, COVERAGE_ITEM: 0.0284623, COVERAGE_ITEM_CORRECT: 0.0165015, COVERAGE_USER: 0.9998535, COVERAGE_USER_CORRECT: 0.9607326, DIVERSITY_GINI: 0.0032162, SHANNON_ENTROPY: 6.2372896, RATIO_DIVERSITY_HERFINDAHL: 0.9807523, RATIO_DIVERSITY_GINI: 0.0128100, RATIO_SHANNON_ENTROPY: 0.5027816, RATIO_AVERAGE_POPULARITY: 3.3437807, RATIO_NOVELTY: 0.0246193, 

SearchBayesianSkopt: Saving model in ../result_experiments/5_S_SLIM_2SLIMElasticN

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 564.1300
Function value obtained: -0.2271
Current minimum: -0.2271
Iteration No: 2 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 0.00046032370443108855, 'topK': 418, 'alpha': 0.5037059824136357, 'workers': 10}


100%|███████████████████████████████████████████████████████████████████████████▉| 18056/18059 [09:28<00:00, 75.61it/s]

  0%|                                                                              | 8/18059 [00:09<6:07:56,  1.22s/it]

  0%|▏                                                                              | 40/18059 [00:09<55:32,  5.41it/s]

  0%|▎                                                                              | 72/18059 [00:13<43:43,  6.86it/s]

  1%|▍                                                                             | 104/18059 [00:13<25:07, 11.91it/s]

  1%|▌                                                                             | 136/18059 [00:14<19:44, 15.13it/s]

  1%|▋                                                                             | 168/18059 [00:15<14:58, 19.91it/s]

  1%|▊                                                                             | 200/18059 [00:15<10:43, 27.76it/s]

  1%|█                          

 24%|██████████████████▎                                                          | 4296/18059 [03:12<09:44, 23.55it/s]

 24%|██████████████████▍                                                          | 4328/18059 [03:15<13:50, 16.53it/s]

 24%|██████████████████▌                                                          | 4360/18059 [03:16<11:53, 19.19it/s]

 24%|██████████████████▋                                                          | 4392/18059 [03:17<10:05, 22.56it/s]

 24%|██████████████████▊                                                          | 4424/18059 [03:20<13:58, 16.27it/s]

 25%|██████████████████▉                                                          | 4456/18059 [03:21<10:05, 22.48it/s]

 25%|███████████████████▏                                                         | 4488/18059 [03:21<07:25, 30.44it/s]

 25%|███████████████████▎                                                         | 4520/18059 [03:21<06:37, 34.10it/s]

KeyboardInterrupt: 

In [4]:
import scipy.sparse as sps
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender

hyperp = {'l1_ratio': 0.01, 'topK': 3000, 'alpha': 0.027207998746246062, 'workers': 8, 'mw': 1}

ICM_genre_subgenre = sps.hstack((ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_length_all_5km))

recommender = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_all, ICM_genre_subgenre)
recommender.fit(**hyperp)

import pandas as pd
at = 10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
sep=",",
dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
	subm_res["user_id"].append(user_id)
	res = recommender.recommend(user_id, cutoff=at)
	res = ' '.join(map(str, res))
	if user_id < 3:
		print(user_id)
		print(res)
	subm_res["item_list"].append(res)


	# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
	# submission

from datetime import datetime

now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)


  0%|                                                                                        | 0/18059 [00:00<?, ?it/s]

(13989, 18059)


100%|███████████████████████████████████████████████████████████████████████████▉| 18032/18059 [16:37<00:01, 24.07it/s]

0
8465 3899 5433 2243 16365 16359 9130 10586 17961 640
1
3473 14400 940 16012 2118 640 10118 9275 9373 16365
2
6177 1551 2846 17022 4607 17330 14229 16365 2748 5925


100%|███████████████████████████████████████████████████████████████████████████▉| 18056/18059 [16:50<00:00, 24.07it/s]